In [1]:
import pennylane as qml
from pennylane import numpy as np
import jax
import optax

In [2]:
dev = qml.device("default.qubit", 4)

@qml.qnode(dev)
def circuit_expected(H):
    qml.BasisState([1,1,0,0], wires=[0, 1, 2, 3])
    qml.DoubleExcitation(0.2, wires=[0, 1, 2, 3])
    return qml.expval(H)

In [6]:
def loss_f(coord):
    symbols = ["H", "H"]
    H_0 = np.array([0.,0.,0.])  # fixed
    H, qb = qml.qchem.molecular_hamiltonian(symbols, np.array([*H_0, *coord]))
    return circuit_expected(H)

In [7]:
def optimize():
    
    H_1 = np.array([1.,1.,1.])
    
    # prepare for the 1st run
    conv_tol = 1e-6
    opt = optax.sgd(learning_rate=0.4)

    max_iterations = 100
    opt = optax.sgd(learning_rate=0.4)
    opt_coords_state = opt.init(H_1)
    
    for i in range(10):        
        grad_coordinates = jax.grad(loss_f, 0)(H_1)
        updates, opt_coords_state = opt.update(grad_coordinates, opt_coords_state)
        H_1 = optax.apply_updates(H_1, updates)
        print(grad_coordinates)
    
optimize()

TracerArrayConversionError: The numpy.ndarray conversion method __array__() was called on traced array with shape float32[].
See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.TracerArrayConversionError